In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
x = np.random.randint(0, 2, 100)
y = np.random.randint(0, 2, 100)

In [ ]:
x.astype(np.float32)
y.astype(np.float32)

In [ ]:
precision = precision_score(x, y, average="micro")
precision

In [ ]:
recall = recall_score(x, y, average="micro")
f1 = f1_score(x, y, average="micro")
(recall, f1)

In [ ]:
from sklearn import metrics
from sklearn.metrics import auc 
import numpy as np
y = np.array([1, 1, 2, 2])  
scores = np.array([0.1, 0.4, 0.35, 0.8])  
fpr, tpr, thresholds = metrics.roc_curve(y, scores, pos_label=2)
metrics.auc(fpr, tpr)

In [ ]:
thresholds

In [ ]:
from sklearn.model_selection import KFold
import numpy as np

# 生成示例数据
X = np.array([[1, 2], [3, 4], [5, 6], [7, 8], [9, 10], [11, 12]])
y = np.array([0, 1, 0, 1, 0, 1])

# 创建 KFold 对象，将数据分成 3 个折
kf = KFold(n_splits=3, shuffle=True, random_state=42)

# 迭代每个折
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    # 在训练集上训练模型（这里做一个简单的打印示例）
    print("Train Set:")
    print("X_train:", X_train)
    print("y_train:", y_train)
    
    # 在测试集上测试模型（这里做一个简单的打印示例）
    print("Test Set:")
    print("X_test:", X_test)
    print("y_test:", y_test)
    
    print("=======================")


In [ ]:
import pandas as pd 

train = pd.read_csv("/Users/zhenhan/Desktop/深度学习/data/房屋数据（2011.8-2015.6）.csv")
train.head()

In [ ]:
train.shape

In [ ]:
stats = []
for col in train.columns:
    stats.append([
        col,
        train[col].nunique(),
        train[col].isna().sum() * 100 / train.shape[0],
        train[col].value_counts(normalize=True, dropna=False).values[0] * 100,
        train[col].dtype
    ])
stats_df = pd.DataFrame(stats, columns=['Feature', 'Unique_values','Missing_values_percent', 'biggest values', 'type'])
stats_df.sort_values('Missing_values_percent', ascending=False)

In [ ]:
stats_df["type"].value_counts(normalize=True)

In [ ]:
missing = train.isnull().sum()
missing = missing[missing > 0]
missing.sort_values(inplace=True)
missing.plot.bar()

In [ ]:
train["price_doc"].describe()

In [ ]:
import seaborn as sen 
import matplotlib.pyplot as plt

plt.figure(figsize=(12,10))
sen.displot(train["price_doc"],  bins=100)

In [ ]:
import numpy as np
import seaborn as sen 
import matplotlib.pyplot as plt

plt.figure(figsize=(12,10))
sen.displot(np.log(train["price_doc"]),  bins=100)

In [ ]:
df_num = train.select_dtypes(include=['float64', 'int64'])
df_num = df_num[df_num.columns.tolist()[::-1][0:4]]
df_num.hist(figsize=(16, 20), bins=50, xlabelsize=8, ylabelsize=8)

In [ ]:
corrmat = train.corr()

f, ax = plt.subplots(figsize=(20,9))
sen.heatmap(corrmat, vmax=0.8)

In [ ]:
corrmat["price_doc"].sort_values(ascending=False)[:10]

In [ ]:
corrmat["price_doc"].sort_values(ascending=False)[::-1][:10]

In [ ]:
import seaborn as sns

df_num = train.select_dtypes(['object'])
df_num = df_num[df_num.columns.to_list()[1:5]]
for col in df_num.columns:
    value_counts = df_num[col].value_counts()
    plt.figure(figsize=(6, 4))
    sns.barplot(x=value_counts.index, y=value_counts.values, palette='Set2')

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset,DataLoader
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

Matplotlib created a temporary config/cache directory at /var/folders/_n/4rn71s9d0kbcfsx7vgnkblrc0000gn/T/matplotlib-9bz6erd9 because the default path (/Users/zhenhan/.matplotlib) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.
Fontconfig warning: ignoring UTF-8: not a valid region tag


In [2]:
train = pd.read_csv("/Users/zhenhan/Desktop/深度学习/data/房屋数据（2011.8-2015.6）.csv")
train_isna = []
for col in train.columns:
    if train[col].isna().sum() / train.shape[0] > 0.4:
        train_isna.append(col)
df = train.drop(train_isna, axis=1)
df_num = df.select_dtypes(include=['int64', 'float64'])
df_obj = df.select_dtypes(include=['object'])
df_num_means = df_num.median()
df_num = df_num.fillna(df_num_means)
df_num.isna().sum().value_counts()
df_num = df_num.iloc[:,1:]
df_obj = df_obj.iloc[:,1:]
df_obj_map = {"poor":0, "no data":2, "satisfactory":2, "good":3, "excellent":4}
df_obj["ecology"] = df_obj["ecology"].map(df_obj_map)
label_encoder = LabelEncoder()
df_obj['sub_area'] = label_encoder.fit_transform(df_obj['sub_area'])
df_obj = pd.get_dummies(df_obj, columns=df_obj.columns[2:14], prefix=df_obj.columns[2:14])
df_obj = pd.get_dummies(df_obj, columns=['product_type'], prefix=['product_type'])
target_column = df_num.columns[-1]
for col in df_num.columns:
    correlation = df[col].corr(df[target_column])
    if abs(correlation) <= 0.05:
        df_num = df_num.drop(col, axis=1)
data = pd.concat([df_num, df_obj], axis=1)
X = data.drop('price_doc', axis=1)
y = data['price_doc']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model_1 = XGBRegressor(n_estimators=500, max_depth=8, learning_rate=0.01, random_state=42)
model_1.fit(X_train, y_train)

y_pre = model_1.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pre))
print(rmse)
r2 = r2_score(y_test, y_pre)
print(r2)

In [ ]:
model_1 = XGBRegressor()

param_dist = {
    'n_estimators': range(100, 1001, 100),
    'max_depth': range(3, 16, 3),
    'learning_rate': np.linspace(0.001, 1, 10),
    'subsample': np.linspace(0.5, 1, 10),
    'colsample_bytree': np.linspace(0.3, 1, 10),
    'min_child_weight': range(1, 6, 1)
}

grid = RandomizedSearchCV(model_1,
                        param_dist,
                        cv=5,
                        scoring = 'neg_mean_squared_error',
                        n_iter=100,
                        n_jobs=8)

grid.fit(X, y)

best_estimator = grid.best_estimator_
print(best_estimator)
print(grid.best_params_)
print(grid.best_score_)

In [ ]:
model_2 = LGBMRegressor()

param_dist = {
    'n_estimators': range(100, 1001, 100),
    'max_depth': range(3, 16, 3),
    'learning_rate': np.linspace(0.001, 1, 10),
    'subsample': np.linspace(0.5, 1, 10),
    'colsample_bytree': np.linspace(0.3, 1, 10),
    'min_child_samples': range(1, 6, 1)  # 修改超参数名称
}

grid = RandomizedSearchCV(
    model_2,
    param_distributions=param_dist,
    cv=3,
    scoring='neg_mean_squared_error',
    n_iter=100,
    n_jobs=8
)

grid.fit(X, y)

best_estimator = grid.best_estimator_
best_params = grid.best_params_
best_score = grid.best_score_

print(best_estimator)
print(best_params)
print(best_score)

In [ ]:
class MLP_Net(nn.Module):
    def __init__(self):
        super(MLP_Net, self).__init__()
        self.fc1 = nn.Linear(256, 512)
        self.relu1 = nn.ReLU()

        self.fc2 = nn.Linear(512, 1024)
        self.relu2 = nn.ReLU()

        self.fc3 = nn.Linear(1024, 1024)
        self.relu3 = nn.ReLU()
        
        self.fc4 = nn.Linear(1024, 256)
        self.relu4 = nn.ReLU()

        self.fc5 = nn.Linear(256, 1)
    
    def forward(self, x):
        outs = self.fc1(x).clamp(min=0) 
        outs = self.relu1(outs)
        
        outs = self.fc2(outs).clamp(min=0) 
        outs = self.relu2(outs)
        
        outs = self.fc3(outs).clamp(min=0) 
        outs = self.relu3(outs)

        outs = self.fc4(outs).clamp(min=0) 
        outs = self.relu4(outs)

        outs = self.fc5(outs)
        
        return outs


In [ ]:
train_loader = DataLoader(TensorDataset(torch.tensor(X_train.values).float(), torch.tensor(y_train.values).float()), batch_size=32, shuffle=True)
val_laoder = DataLoader(TensorDataset(torch.tensor(X_test.values).float(), torch.tensor(y_test.values).float()), batch_size=32, shuffle=False)

model_3 = MLP_Net()

loss_func = nn.SmoothL1Loss()
optimizer = torch.optim.AdamW(params=model_3.parameters(), lr = 0.01)

def metric_func(y_pred, y_true):
    y_pred = y_pred.detach().cpu().numpy()
    y_true = y_true.detach().cpu().numpy()
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    return rmse, r2


In [ ]:
import datetime

epochs = 20
log_step_freq = 300
dishistory = pd.DataFrame(columns= ["epochs", "train_loss", "val_loss", "Train_rmse", "Train_R^2","val_rmse", "val_R^2"])
print("Strat Training")
nowtime = datetime.datetime.now().strftime("%H:%M:%S")
print("=========="*8 + "%s"%nowtime)

for epoch in range(1, epochs + 1):
    model_3.train()
    loss_sum = 0.
    rmse_sum = 0.
    r2_sum = 0.
    step = 1

    for step, (feature, target) in enumerate(train_loader, 1):
        optimizer.zero_grad()
        predictions = model_3(feature)
        loss = loss_func(predictions, target.view(-1, 1))
        rmse, r2  = metric_func(predictions, target.view(-1, 1))
        loss.backward()
        optimizer.step()
        loss_sum += loss.item()
        rmse_sum += rmse.item()
        r2_sum += r2.item()

        if step%log_step_freq == 0:
            print("[step = %d] loss: %d, rmse: %d, r2: %.3f" % (step, loss_sum/step, rmse_sum/step, r2_sum/step))

    model_3.eval()
    val_loss_sum = 0.
    val_rmse_sum = 0.
    val_r2_sum = 0.
    val_step = 1
    best_val_loss = float('inf')
    best_model_state_dict = None

    for val_step, (feature, target) in enumerate(val_laoder, 1):
        predictions = model_3(feature)
        val_loss = loss_func(predictions, target.view(-1,1))
        val_rmse, val_r2 = metric_func(predictions, target)

        val_loss_sum += val_loss.item()
        val_rmse_sum += val_rmse.item()
        val_r2_sum += val_r2.item()

        # if val_step%log_step_freq == 0:
        #     print("[step = %d] loss: %d, rmse: %d, r2: %.3f") % (step, val_loss_sum/step, val_rmse_sum/step, val_r2_sum/step)
    

    info = (epoch, loss_sum/step, val_loss_sum/val_step, rmse_sum/step, r2_sum/step, val_rmse_sum/val_step, val_r2/val_step)
    dishistory.loc[epoch-1] = info
    if (val_loss_sum / val_step) < best_val_loss:
        best_val_loss = val_loss_sum / val_step
        best_model_state_dict = model_3.state_dict()
    
    torch.save(best_model_state_dict, 'best_model.pth')

    print("[EPOCH = %d] trian_loss = %d, val_loss = %d, trian_rmse =  %d, train_r2 = %.3f, val_rmse = %d, val_r2 = %.3f" % info)
    nowtime = datetime.datetime.now().strftime('%H:%M:%S')
    print("\n"+"=========="*8 + "%s"%nowtime)
print('Finished training')

In [ ]:
model_3 = MLP_Net()
model_3.load_state_dict(torch.load('./best_model.pth'))
model_3.eval()

predictions = []
targets = []

with torch.no_grad():
    for step, (feature, target) in enumerate(val_laoder):
        output = model_3(feature)
        predictions.append(output.cpu().numpy())
        targets.append(target.cpu().numpy())

predictions = np.concatenate(predictions)
targets = np.concatenate(targets)
# print(predictions)
rmse = np.sqrt(mean_squared_error(targets, predictions))
r2 = r2_score(targets, predictions)

print("RMSE:", rmse)
print("R^2:", r2)

In [ ]:
df = dishistory
x = df["epochs"]
y_columns = ["train_loss", "val_loss", "Train_rmse", "Train_R^2", "val_rmse", "val_R^2"]
y = df[y_columns]
plt.figure(figsize=(10, 6))

for column in y_columns:
    plt.plot(x, df[column], marker='o', label=column)
plt.xlabel("Epochs")
plt.ylabel("Values")
plt.title("Training and Validation Metrics")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# 导入所需的库
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error

# 初始化不同的回归器
# reg1 = SVR()
# reg2 = RandomForestRegressor()
reg3 = GradientBoostingRegressor()
reg4 = XGBRegressor()
reg5 = LGBMRegressor()
reg6 = CatBoostRegressor()

# 训练并预测
for reg in [reg3, reg4, reg5, reg6]:
    reg.fit(X_train, y_train)
    y_pred = reg.predict(X_test)
    mse = np.sqrt(mean_squared_error(y_test,y_pred))
    r2 = r2_score(y_test, y_pred)
    print(f"{reg.__class__.__name__} MSE: {mse:.4f}")
    print(f"{reg.__class__.__name__} R2: {r2:.4f}")


In [ ]:
from sklearn.ensemble import VotingRegressor

# 初始化不同的回归器
# reg1 = SVR()
# reg2 = RandomForestRegressor()
reg3 = GradientBoostingRegressor()
reg4 = XGBRegressor()
reg5 = LGBMRegressor()
reg6 = CatBoostRegressor()

# 将回归器放入列表中
regressors = [
            #   ('SVR', reg1),
            #   ('Random Forest', reg2),
              ('Gradient Boosting', reg3),
              ('XGBoost', reg4),
              ('LGBoost', reg5),
              ('CATBoost', reg6)]

# 初始化投票回归器
voting_reg = VotingRegressor(estimators=regressors)

# 训练投票回归器
voting_reg.fit(X_train, y_train)
y_pred = voting_reg.predict(X_test)
mse = np.sqrt(mean_squared_error(y_test,y_pred))
r2 = r2_score(y_test, y_pred)

In [6]:
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from catboost import CatBoostRegressor

lgb_reg = lgb.LGBMRegressor()
# lgb_reg = XGBRegressor()
# lgb_reg = CatBoostRegressor()
# 定义要调优的参数和对应的候选值
param_grid = {
    'learning_rate': [0.02],
    'max_depth': [10],
    'n_estimators': [900],
    'subsample': [0.5]
}

# 初始化网格搜索对象
grid_search = GridSearchCV(estimator=lgb_reg, param_grid=param_grid, scoring='neg_mean_squared_error', cv=3, verbose=2, n_jobs=8)

# 在训练集上进行网格搜索调参
grid_search.fit(X_train, y_train)

# 输出最优参数和最优模型
print("Best parameters found: ", grid_search.best_params_)
print("Best negative mean squared error found: ", grid_search.best_score_)

# 使用最优参数的模型进行预测
best_lgb_reg = grid_search.best_estimator_
y_pred = best_lgb_reg.predict(X_test)

# 计算均方误差
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Mean Squared Error on test set: ", rmse)
r2 = r2_score(y_test, y_pred)
print("R2 Error on test set: ", r2)


Fitting 3 folds for each of 1 candidates, totalling 3 fits
[CV] END learning_rate=0.02, max_depth=10, n_estimators=900, subsample=0.5; total time=   9.4s
[CV] END learning_rate=0.02, max_depth=10, n_estimators=900, subsample=0.5; total time=   9.5s
[CV] END learning_rate=0.02, max_depth=10, n_estimators=900, subsample=0.5; total time=   9.6s


: 

: 